In [1]:
from utils import time_op

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy
from formulas import GPMC, Formula, BuddyNode, BuddyContext, random_k_cnf

n,m,k = 8, 10, 7
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

solver = GPMC()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

with BuddyContext(vars) as ctx:
    f = time_op(ctx.parse)(formula)
    satcount_bdd = time_op(f.satcount)()
    print(f"BDD satcount = {satcount_bdd}")


[00006.0134 ms / 0.0060 s / 0.0001 min] satcount
GPMC satcount = 244
[00082.0720 ms / 0.0821 s / 0.0014 min] parse
[00000.0036 ms / 0.0000 s / 0.0000 min] satcount
BDD satcount = 244.0


In [3]:
n,clauses_to_variables,k = 20, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import impmeas.bdds as bdd 
from formulas import BuddyContext

with BuddyContext(vars) as ctx:
    x = "x1"
    f = time_op(ctx.parse)(formula)
    print(f"bdd size = {f.nodecount} with satcount = {f.satcount()}")
    blame, ub = time_op(bdd.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(bdd.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[01738.1699 ms / 1.7382 s / 0.0290 min] parse
bdd size = 11143 with satcount = 419578.0
=== COMPUTING BLAME for x1 in BDD with size 11143 ===
k=0 size ell=8437 d result=0.2053 max increase possible=0.3974 
k=1 size ell=39016 d result=0.2060 max increase possible=0.1276 
k=2 size ell=18940 d result=0.1095 max increase possible=0.0136 
k=3 size ell=1488 d result=0.0133 max increase possible=0.0002 
k=4 size ell=20 d result=0.0002 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5343, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[00503.3259 ms / 0.5033 s / 0.0084 min] blame
[00000.0312 ms / 0.0000 s / 0.0000 min] influence
influence 0.20529, 0.53429 <= blame <= 0.53430, blame diff 0.00000


In [5]:
import impmeas.mc as mc
from formulas import GPMC, Formula, FormulaContext

solver = GPMC(cs=20000)
ctx = FormulaContext(solver)
f = time_op(ctx.parse)(formula)
x = "x1"
infl = (time_op)(mc.influence)(f, x)
blame, ub = time_op(mc.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[01937.2869 ms / 1.9373 s / 0.0323 min] parse
[00400.9767 ms / 0.4010 s / 0.0067 min] influence
=== COMPUTING BLAME for x1 in Formula with size 3791 ===
k=0 size of cnf: 4298 d result=0.2053 max increase possible=0.3974 
k=1 size of cnf: 4369 d result=0.2060 max increase possible=0.1276 
k=2 size of cnf: 4436 d result=0.1095 max increase possible=0.0136 
k=3 size of cnf: 4499 d result=0.0133 max increase possible=0.0002 
k=4 size of cnf: 4558 d result=0.0002 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5343, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[238086.8673 ms / 238.0869 s / 3.9681 min] blame
influence 0.20529, 0.53429 <= blame <= 0.53430, blame diff 0.00000


In [8]:
import importance_measures.mc as mc 
from formulas import Formula, GPMC

# f,S = mc.at_most(2, {"x1","x2","x3", "x4"})
f,S = mc.at_most_cnf(3, {1,2,3,4}, 5)
print(f)
GPMC().satcount( f )

[[7, -1], [-7, 1], [2, 7, -5], [-2, 5], [-7, 5], [3, 5, -11], [-3, 11], [-5, 11], [4, 11, -9], [-4, 9], [-11, 9], [8, -2, -7], [-8, 7], [-8, 2], [-6, 5, 8], [-6, 3, 8], [6, -3, -5], [6, -8], [-14, 11, 6], [-14, 4, 6], [14, -4, -11], [14, -6], [13, -3, -8], [-13, 8], [-13, 3], [-10, 6, 13], [-10, 4, 13], [10, -4, -6], [10, -13], [12, -4, -13], [-12, 13], [-12, 4], [-12]]


15